# Migrate to SDN4CoRE
This script translates the original NIDSDataSetCreation car network based on INET 4.5 to the SDN4CoRE simulation models.

TODOList:
- [x] Migrate the car network to SDN4CoRE
- [x] Migrate apps to services
- [x] Migrate Time Sync to dummy sync
- [ ] Migrate QoS to SDN4CoRE
  - [x] PCP for all services
  - [x] Static Services for TDMA
  - [x] Service use local clock for send interval (period 1ms, actiontime = 0)
  - [x] GCL for TDMA
    - [x] Guardband 12,272us for fullsize frame with ifg
    - [x] Slots
  - [x] CMI for Shaped Priorities
  - [ ] check reservation for CBS

## Migrate Apps

In [1]:
stopAfterLine = "*.connectivityGateway.app[0].connection[0].source.productionInterval = uniform(10ms,100ms)"

In [2]:
skipSections = { # from: until
    "# Common Ethernet Configuration": "# Application Configuration",
}

In [3]:
insertBeforeLine= {
    "# Application Configuration": [
        "# Common arp setting",
        "**.arpType = \"GlobalARP\"",
        "",
        "#################################",
        "# Clock Configuration",
        "# Tick length 80ns",
        "**.scheduler.tick = 80ns",
        "**.scheduler.period[0].cycle_ticks = sec_to_tick(1ms)",
        " ",
        "# Maximum clock drift of 200ppm per tick",
        "**.scheduler.oscillator.max_drift = 200ppm",
        "# Maximum clock drift change of +-0.1ps per cycle",
        "**.scheduler.oscillator.drift_change =  uniform(-100ppm, 100ppm)",
        "",
        "####################################",
        "# Time Synchronization Configuration",
        "# uses DummySync, no on wire protocol, e.g., gptp",
        "# precission of synchronisation",
        "**.precission = 500ns",
        "", 
        "###########################",
        "# Shaper Configuration",
        "include shaping.ini # written by hand",
        "",
        "###########################",
        "# Conntroller Configuration",
        "include controller.ini # written by hand",
        "",
        "###########################",
        "# Study Configuration",
        "include studies.ini # written by hand",
        "",
        "###########################"
    ],
    "# camera applications": [
        "**.services[*].applicationUDPPort = 1000 + index",
        "**.services[*].startTime = 0.09s",
        " "
    ]
}

In [4]:
# see readme for traffic description.
pcpForName = {
    "AutoThrottle": 6,
    "AutoBrake": 6,
    "AutoSteer": 6,
    "ManualThrottle": 6,
    "ManualBrake": 6,
    "ManualSteer": 6,
    "Video": 5,
    "Lidar": 5,
    "Control": 4,
    "EtsiCam": 2,
}
def getPCPForName(line):
    for name in pcpForName:
        if name in line:
            return pcpForName[name]
    print("No PCP for name found in line: ", line)
    return 0

In [5]:
pubIds = []
subIds = []
isTcpClient = False

def isAppConfig(line):
    return ".app[" in line

def parseUniform(line):
    return line.split("=")[1].replace("uniform(", "").replace(")", "").strip().split(",")

def parseTimeWithUnit(valueWithUnit):
    cleanVWU = valueWithUnit.strip()
    if "ns" in cleanVWU:
        return float(cleanVWU.replace("ns", ""))/(10 ** 9)
    elif "us" in cleanVWU:
        return float(cleanVWU.replace("us", ""))/(10 ** 6)
    elif "ms" in cleanVWU:
        return float(cleanVWU.replace("ms", ""))/(10 ** 3)
    return float(cleanVWU.replace("s", ""))

def parseLenghtWithUnit(valueWithUnit):
    cleanVWU = valueWithUnit.strip()
    return float(cleanVWU.replace("b", ""))

def translateAppConfig(line):
    global isTcpClient
    result = []
    line = line.replace(".app[", ".services[")
    line = line.replace(".connection[0]","")
    if "packetNameFormat" in line:
        node = line.split(".")[1].strip()
        format = line.split("=")[1].replace("%M", node).replace("-%c", "").strip()
        result.append(line.split("=")[0].strip().replace("source.packetNameFormat", "display-name") + " = " + format)
        result.append(line.split("=")[0].strip().replace("source.packetNameFormat", "serviceName") + " = " + format)
        pcp = getPCPForName(line)
        if pcp > 0:
            result.append(line.split("=")[0].strip().replace("source.packetNameFormat", "pcp") + " = " + str(pcp))
        if pcp == 6:
            result.append(line.split("=")[0].strip().replace("source.packetNameFormat", "isStatic") + " = true")
    elif "display-name" in line:
        if "sink" in line:
            result.append(line)
            pcp = getPCPForName(line)
            if pcp == 6:
                result.append(line.split("=")[0].strip().replace("display-name", "isStatic") + " = true")
    elif "typename" in line:
        isTcpClient = False
        if "UdpSourceApp" in line:
            if "zonalControllerFrontLeft.services[0..2]" in line or "adas.services[6..8]" in line:
                result.append(line.replace("UdpSourceApp", "SyncPublisher"))
            else:
                result.append(line.replace("UdpSourceApp", "Publisher"))
        elif "UdpSinkApp" in line:
            result.append(line.replace("UdpSinkApp", "Subscriber"))
        elif "TcpClientApp" in line:
            isTcpClient = True
            result.append(line.replace("TcpClientApp", "Subscriber"))
            result.append(line.split("=")[0].strip().replace("typename", "qosGroup") + " = \"SOMEIP_TCP\"")
        elif "TcpServerApp" in line:
            result.append(line.replace("TcpServerApp", "Publisher"))
            result.append(line.split("=")[0].strip().replace("typename", "qosGroups") + " = \"SOMEIP_TCP\"")
        else:
            result.append(line)
    elif "productionInterval" in line:
        if "uniform" in line:
            interval = parseUniform(line)
            intervalValues = [parseTimeWithUnit(interval[0]), parseTimeWithUnit(interval[1])]
            parameter = line.split("=")[0].strip().replace("source.productionInterval", "intervalMin")
            if intervalValues[0] == intervalValues[1]:
                result.append(parameter + " = " + interval[0])
            elif intervalValues[0] > intervalValues[1]:
                result.append(parameter + " = " + interval[1])
                result.append(parameter.replace("Min", "Max") + " = " + interval[0])
            else: # intervalValues[0] < intervalValues[1]
                result.append(parameter + " = " + interval[0])
                result.append(parameter.replace("Min", "Max") + " = " + interval[1])
        else:
            result.append(line.replace("source.productionInterval", "intervalMin"))
    elif "packetLength" in line:
        if "uniform" in line:
            length = parseUniform(line)
            lengthValues = [parseLenghtWithUnit(length[0]), parseLenghtWithUnit(length[1])]
            parameter = line.split("=")[0].strip().replace("source.packetLength", "payloadMax")
            if lengthValues[0] == lengthValues[1]:
                result.append(parameter + " = " + length[0])
            elif lengthValues[0] > lengthValues[1]:
                result.append(parameter + " = " + length[0])
                result.append(parameter.replace("Max", "Min") + " = " + length[1])
            else: # lengthValues[0] < lengthValues[1]
                result.append(parameter + " = " + length[1])
                result.append(parameter.replace("Max", "Min") + " = " + length[0])
        else:
            result.append(line.replace("source.packetLength", "payloadMax"))
    elif "destAddress" in line:
        if "224." in line:
            result.append(line.replace("io.destAddress", "mcastDestAddress"))
            result.append(line.split("=")[0].strip().replace("io.destAddress", "qosGroups") + " = \"SOMEIP_UDP_MCAST\"")
    elif "multicastAddresses" in line:
        result.append(line.split("=")[0].strip().replace("io.multicastAddresses", "qosGroup") + " = \"SOMEIP_UDP_MCAST\"")
        address = line.split("=")[1].replace("[", "").replace("]", "").strip()
        result.append(line.replace("io.multicastAddresses", "mcastDestAddress").split("=")[0].strip() + " = " + address)
    elif "connectAddress" in line:
        pass
    elif "destPort" in line:
        result.append(line.replace("io.destPort", "mcastDestPort"))
        result.append(line.replace("io.destPort", "serviceId"))
        serviceId = line.split("=")[1].strip()
        if serviceId not in pubIds:
            pubIds.append(serviceId)
        else:
            print("ServiceId already exists: " + serviceId + " in line: " + line)
    elif "listener.localPort" in line:
        result.append(line.replace("listener.localPort", "applicationTCPPort"))
        result.append(line.replace("listener.localPort", "serviceId"))
        serviceId = line.split("=")[1].strip()
        if serviceId not in pubIds:
            pubIds.append(serviceId)
        else:
            print("ServiceId already exists: " + serviceId + " in line: " + line)
    elif "io.localPort" in line:
        if isTcpClient:
            result.append(line.replace("io.localPort", "applicationTCPPort"))
        else:
            result.append(line.replace("io.localPort", "serviceId"))
            result.append(line.replace("io.localPort", "mcastDestPort"))
            serviceId = line.split("=")[1].strip()
            if serviceId not in subIds:
                subIds.append(serviceId)
    elif "connectPort" in line:
        result.append(line.replace("io.connectPort", "serviceId"))
        serviceId = line.split("=")[1].strip()
        if serviceId not in subIds:
            subIds.append(serviceId)
        else:
            print("Unicast Sub ServiceId already exists: " + serviceId + " in line: " + line)
    else:
        result.append(line)
    return result

def translate(line):
    if line[0] == "#":
        return [line]
    comment = ""
    if "#" in line:
        comment = line.split("#")[1]
        line = line.split("#")[0]
    result = []
    if isAppConfig(line):
        newlines = translateAppConfig(line)
        for newline in newlines:
            result.append(newline)
    elif "numApps" in line:
        result.append(line.replace("numApps", "numServices"))
    else:
        result.append(line)
    if comment != "":
        for i in range(result.__len__()):
            result[i] = result[i].split("#")[0]
            if(i == 0):
                result[0] = result[0] + " #" + comment
    return result


In [6]:
DONT_SKIP = "THIS TEXT WILL NEVER OCCUR XD"
def translateFile(infile, outfile):
    stop = False
    lastLine = ""
    skipUntil = DONT_SKIP
    for line in infile:
        if line == "\n" and lastLine == "\n":
            continue
        if stopAfterLine in line:
            stop = True
        for insertBefore in insertBeforeLine.keys():
            if insertBefore in line:
                for newline in insertBeforeLine[insertBefore]:
                    outfile.write(newline + "\n")
        for skipLine in skipSections.keys():
            if skipLine in line:
                skipUntil = skipSections[skipLine] 
                break
        if skipUntil != DONT_SKIP:
            if skipUntil in line:
                skipUntil = DONT_SKIP
            else:
                continue
        newlines = translate(line)
        for newline in newlines:
            if "\n" not in newline:
                newline = newline + "\n"
            outfile.write(newline)
        if stop:
            break
        lastLine = line
    infile.close()
    outfile.close()

In [7]:
# read the ini file to modify
files = [
    "omnetpp.ini",
    "secvi_car_infotainment.ini",
    "secvi_car_zonalControllerFrontLeft.ini",
    "secvi_car_zonalControllerFrontRight.ini",
    "secvi_car_zonalControllerRearLeft.ini",
    "secvi_car_zonalControllerRearRight.ini"
]
for file in files:
    infile = open("originals/" + file, "r")
    outfile = open(file, "w")
    translateFile(infile, outfile)

In [8]:
# verify service ids
unused_subIds = [id for id in subIds if id not in pubIds]
for id in pubIds:
    if id not in subIds:
        print("ServiceId " + id + " is not used by any subscriber")
for id in unused_subIds:
    print("ServiceId " + id + " is not used by any publisher")
